Tutorial: Building a Knowledge-Enhanced Chatbot with LangChain and Pinecone

Resources
Before you start, ensure you have the following:

Python 3.7 or higher installed on your system.
An OpenAI API key.
A Pinecone API key.
The dataset of Arvix papers (chunks_articles.csv).


Install the required libraries using pip:

pip install -qU langchain openai pinecone-client tiktoken langchain-community langsmith typing_extensions


1. Introduction and Setup
To begin, we'll create a simple chatbot without any retrieval augmentation by initializing a ChatOpenAI object. Ensure you have your OpenAI API key ready.

2. Loading and Preparing the Dataset
Load the dataset of Arvix papers from a local CSV file (see file above) using pandas:

In [23]:
import pandas as pd

# Load the dataset
dataset = pd.read_csv("chunks_metadata.csv")

# Display the first few rows of the dataset
dataset.head()

,chunk_id,chunk,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,2401.08396_0,Hidden Flaws Behind Expert-Level Accuracy of ...,2401.08396,Qiao Jin,"Qiao Jin, Fangyuan Chen, Yiliang Zhou, Ziyang ...",Hidden Flaws Behind Expert-Level Accuracy of G...,Under review,NaN,NaN,NaN,cs.CV cs.AI cs.CL,http://creativecommons.org/licenses/by/4.0/,Recent studies indicate that Generative Pre-...,"[{'version': 'v1', 'created': 'Tue, 16 Jan 202...",2024-04-24,"[['Jin', 'Qiao', ''], ['Chen', 'Fangyuan', '']..."
1,2401.08396_1,g such multimodal AI models into clinical work...,2401.08396,Qiao Jin,"Qiao Jin, Fangyuan Chen, Yiliang Zhou, Ziyang ...",Hidden Flaws Behind Expert-Level Accuracy of G...,Under review,NaN,NaN,NaN,cs.CV cs.AI cs.CL,http://creativecommons.org/licenses/by/4.0/,Recent studies indicate that Generative Pre-...,"[{'version': 'v1', 'created': 'Tue, 16 Jan 202...",2024-04-24,"[['Jin', 'Qiao', ''], ['Chen', 'Fangyuan', '']..."
2,2401.08396_2,"es the correct final choices (35.5%), most pro...",2401.08396,Qiao Jin,"Qiao Jin, Fangyuan Chen, Yiliang Zhou, Ziyang ...",Hidden Flaws Behind Expert-Level Accuracy of G...,Under review,NaN,NaN,NaN,cs.CV cs.AI cs.CL,http://creativecommons.org/licenses/by/4.0/,Recent studies indicate that Generative Pre-...,"[{'version': 'v1', 'created': 'Tue, 16 Jan 202...",2024-04-24,"[['Jin', 'Qiao', ''], ['Chen', 'Fangyuan', '']..."
3,2401.08396_3,"Bethesda, MD, USA. 8Department of Neurology,...",2401.08396,Qiao Jin,"Qiao Jin, Fangyuan Chen, Yiliang Zhou, Ziyang ...",Hidden Flaws Behind Expert-Level Accuracy of G...,Under review,NaN,NaN,NaN,cs.CV cs.AI cs.CL,http://creativecommons.org/licenses/by/4.0/,Recent studies indicate that Generative Pre-...,"[{'version': 'v1', 'created': 'Tue, 16 Jan 202...",2024-04-24,"[['Jin', 'Qiao', ''], ['Chen', 'Fangyuan', '']..."
4,2401.08396_4,"an Peng, Ph.D., FAMIA Assistant Professor Depa...",2401.08396,Qiao Jin,"Qiao Jin, Fangyuan Chen, Yiliang Zhou, Ziyang ...",Hidden Flaws Behind Expert-Level Accuracy of G...,Under review,NaN,NaN,NaN,cs.CV cs.AI cs.CL,http://creativecommons.org/licenses/by/4.0/,Recent studies indicate that Generative Pre-...,"[{'version': 'v1', 'created': 'Tue, 16 Jan 202...",2024-04-24,"[['Jin', 'Qiao', ''], ['Chen', 'Fangyuan', '']..."


3. Building the Knowledge Base
Initializing Pinecone
Set up your Pinecone API key and initialize the Pinecone client:

In [24]:
from pinecone import Pinecone
import os

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.getenv("PINECONE_API_KEY") or ""

# configure client
pc = Pinecone(api_key=api_key)

Setting Up the Index Specification
Configure the cloud provider and region for your index:

In [25]:
from pinecone import ServerlessSpec

spec = ServerlessSpec(cloud="aws", region="us-east-1")

Initializing the Index
Create and initialize the index if it doesn't already exist:

In [26]:
import time

index_name = 'rag'
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

# check if index already exists
if index_name not in existing_indexes:
    # create index
    pc.create_index(index_name, dimension=1536, metric='dotproduct', spec=spec)
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4860}},
 'total_vector_count': 4860}

4. Creating Embeddings and Populating the Index
Instantiating the Embeddings Model
Set up OpenAI's embedding model via LangChain:

In [27]:
from langchain.embeddings import OpenAIEmbeddings
import os

# Ensure you have your OpenAI API key set in your environment variables
os.environ["OPENAI_API_KEY"] = ""

# Instantiate the embeddings model
embed_model = OpenAIEmbeddings(model="text-embedding-3-small")

Embedding Text Data
Generate embeddings for sample text data:


In [28]:
texts = ['this is the first chunk of text', 'then another second chunk of text is here']

res = embed_model.embed_documents(texts)
len(res), len(res[0])

(2, 1536)

Embedding and Indexing the Dataset
Embed and insert data into Pinecone in batches:

In [29]:
from tqdm.auto import tqdm
import pandas as pd

data = dataset  # this makes it easier to iterate over the dataset
batch_size = 30

for i in tqdm(range(0, len(data), batch_size)):
    i_end = min(len(data), i + batch_size)
    batch = data.iloc[i:i_end]
    ids = [f"{x['id']}-{x['chunk_id']}" for _, x in batch.iterrows()]
    texts = [str(x['chunk']) for _, x in batch.iterrows()]
    embeds = embed_model.embed_documents(texts)
    metadata = [{'text': str(x['chunk']), 'source': str(x['authors']), 'title': str(x['title'])} for _, x in batch.iterrows()]
    index.upsert(vectors=list(zip(ids, embeds, metadata)))

100%|██████████| 4/4 [00:12<00:00,  3.03s/it]


5. Retrieval Augmented Generation (RAG)
Initializing the Vector Store
Set up LangChain's vectorstore with our Pinecone index:

In [30]:
from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = Pinecone(index, embed_model.embed_query, text_field)

c:\Python312\Lib\site-packages\langchain_community\vectorstores\pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


Querying the Index
Perform a similarity search to retrieve relevant information:

In [31]:
query = "What is GPT-4 Vision ?"
vectorstore.similarity_search(query, k=3)

[Document(page_content="Hidden Flaws Behind Expert-Level Accuracy of  Multimodal GPT-4 Vision in Medicine  Qiao Jin, M.D.1, Fangyuan Chen2, Yiliang Zhou, M.S.3, Ziyang Xu, M.D., Ph.D.4, Justin M. Cheung, M.D.5, Robert Chen, M.D.6, Ronald M. Summers, M.D., Ph.D.7, Justin F. Rousseau, M.D., M.M.Sc.8, Peiyun Ni, M.D.9, Marc J Landsman, M.D.10, Sally L. Baxter, M.D., M.Sc.11, Subhi J. Al'Aref, M.D.12, Yijia Li, M.D.13, Alex Chen14, M.D., Josef A. Brejt14, M.D., Michael F. Chiang, M.D15, Yifan Peng, Ph.D.3,* and Zhiyong Lu, Ph.D.1,*  Brief Abstract (70 words) We conducted a comprehensive evaluation of GPT-4V’s rationales when solving NEJM Image Challenges. We show that GPT-4V achieves comparable results to physicians regarding multi-choice accuracy (81.6% vs. 77.8%). However, we discovered that GPT-4V frequently presents flawed rationales in cases where it makes the correct final choices (35.5%), mostly in image comprehension. As such, our findings emphasize the necessity for in-depth evalu

Augmenting the Prompt
Create a function to augment the chatbot's prompt with retrieved information:

In [32]:
def augment_prompt(query: str):
    results = vectorstore.similarity_search(query, k=3)
    source_knowledge = "\n".join([x.page_content for x in results])
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

# Example usage
print(augment_prompt(query))

Using the contexts below, answer the query.

    Contexts:
    Hidden Flaws Behind Expert-Level Accuracy of  Multimodal GPT-4 Vision in Medicine  Qiao Jin, M.D.1, Fangyuan Chen2, Yiliang Zhou, M.S.3, Ziyang Xu, M.D., Ph.D.4, Justin M. Cheung, M.D.5, Robert Chen, M.D.6, Ronald M. Summers, M.D., Ph.D.7, Justin F. Rousseau, M.D., M.M.Sc.8, Peiyun Ni, M.D.9, Marc J Landsman, M.D.10, Sally L. Baxter, M.D., M.Sc.11, Subhi J. Al'Aref, M.D.12, Yijia Li, M.D.13, Alex Chen14, M.D., Josef A. Brejt14, M.D., Michael F. Chiang, M.D15, Yifan Peng, Ph.D.3,* and Zhiyong Lu, Ph.D.1,*  Brief Abstract (70 words) We conducted a comprehensive evaluation of GPT-4V’s rationales when solving NEJM Image Challenges. We show that GPT-4V achieves comparable results to physicians regarding multi-choice accuracy (81.6% vs. 77.8%). However, we discovered that GPT-4V frequently presents flawed rationales in cases where it makes the correct final choices (35.5%), mostly in image comprehension. As such, our findings emp

6. Let's try a query whitout RAG

In [33]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

messages = [
    SystemMessage(content="You are an expert in the field of AI."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand Recursice Neural Networks.")
]
chat = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model='gpt-3.5-turbo')

# Asking the same question with no RAG
prompt = HumanMessage(content="What is GPT-4 Vision?")
res = chat(messages + [prompt])
print(res.content)

As of my last update, there is no information available about a specific model called "GPT-4 Vision." It is possible that newer models have been developed since then that I am not aware of. If you can provide more context or details, I can try to help you understand or provide information based on the latest available knowledge.


7. Integrating with the Chatbot
Connect the augmented prompt to the chatbot:

In [34]:
# create a new user prompt and let's try with RAG this time 
prompt = HumanMessage(content=augment_prompt(query))
messages.append(prompt)

res = chat(messages)
print(res.content)

GPT-4 Vision, also known as GPT-4V, is a state-of-the-art multimodal Large Language Model (LLM) developed by OpenAI. It allows users to analyze both images and texts together, enabling applications in various domains, including medicine. GPT-4V has been evaluated for its performance in answering multi-choice medical questions, where it has shown to outperform medical students and even physicians in closed-book settings.


Conclusion
By following this tutorial, you've learned how to build a knowledge-enhanced chatbot that leverages a robust knowledge base using LangChain and Pinecone. This setup allows the chatbot to provide more accurate and contextually relevant responses by retrieving and integrating information from the knowledge base.

Experiment with different datasets, queries, and embeddings to further enhance your chatbot's capabilities. Happy coding!